In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.fft import dct, idct
from scipy.signal import convolve2d
from skimage.io import imread

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
rootfolder = ".."

# Denoising

The goal of this section is to implement a simple denoising algorithm based on the 2D DCT. Given a noise free image $Y$, we observe a noisy version $S$:

$$
S = Y + \eta
$$

where $\eta\sim N(0, \sigma^2)$ denotes white Gaussian noise.

Our goal is to compute an estimate $\widehat Y$ of the original image $Y$. To evaluate the performance of the denoising algorithm we use again the PSNR:

$$
\text{PSNR} = 10\log_{10}\frac{1}{\text{MSE}(Y, \widehat Y)}
$$


## Synthetically corrupt an noisy image


Load the image and rescale it in $[0,1]$


In [ ]:
img = imread(f"{rootfolder}/data/cameraman.png") / 255  # /data/checkerboard.png
imsz = img.shape

Corrupt the image with white gaussian noise


In [ ]:
sigma_noise = 20 / 255
noisy_img = img + np.random.normal(size=imsz) * sigma_noise

Compute the psnr of the noisy input


In [ ]:
psnr_noisy = 10 * np.log10(1 / np.mean((noisy_img - img) ** 2))
psnr_noisy

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img, cmap="gray")
ax[0].set_title("Original image")

ax[1].imshow(noisy_img, cmap="gray")
ax[1].set_title(f"Noisy image, PSNR = {psnr_noisy:.2f}")

## Noise estimation

Compute the horizontal derivative of the image


In [ ]:
differences = np.diff(noisy_img, axis=1)

Compute sigma as the empirical std


In [ ]:
sigma_hat_emp = np.std(differences)

Use MAD to estimate the noise level sigma


In [ ]:
sigma_hat = np.median(np.abs(differences - np.median(differences))) / (
    0.6745 * np.sqrt(2)
)

In [ ]:
print(
    f"sigma: {sigma_noise:.3f}, sigma_hat (empirical std): {sigma_hat_emp:.3f}, sigma_hat (MAD): {sigma_hat:.3f}"
)

## Denoising by Smoothing

Implement Denoising by Smoothing using convolution against a uniform filter of different size.


In [ ]:
filter_size = 3

filter = np.ones((filter_size, filter_size)) / (filter_size**2)

# compute the convolution with convolve2d()
img_hat_conv = convolve2d(noisy_img, filter, mode="same", boundary="fill")

In [ ]:
psnr_conv = 10 * np.log10(1 / np.mean((img_hat_conv - img) ** 2))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 10))
ax[0].imshow(img, cmap="gray")
ax[0].set_title("Original image")

ax[1].imshow(img_hat_conv, cmap="gray")
ax[1].set_title(
    f"Image denoised by convolution (filter size: {filter_size}), PSNR = {psnr_conv:.2f}"
)

ax[2].imshow(noisy_img, cmap="gray")
ax[2].set_title(f"Noisy image, PSNR = {psnr_noisy:.2f}")

## Denoising by Leveraging Sparsity in the DCT Domain

Definition of dct2 and idct2 (they are not builtin functions)


In [ ]:
def dct2(s):
    return dct(dct(s.T, norm="ortho").T, norm="ortho")


def idct2(x):
    return idct(idct(x.T, norm="ortho").T, norm="ortho")

In [ ]:
# patch size
p = 8

# number of elements in the patch
M = p**2

Useful function for plot the 2D DCT dictionary


In [ ]:
def get_dictionary_img(D):
    M = D.shape[0]
    p = int(round(np.sqrt(M)))
    bound = 2
    img = np.ones((p * p + bound * (p - 1), p * p + bound * (p - 1)))
    for i in range(M):
        m = np.mod(i, p)
        n = int((i - m) / p)
        m = m * p + bound * m
        n = n * p + bound * n
        atom = D[:, i].reshape((p, p))
        if atom.min() < atom.max():
            atom = (atom - atom.min()) / (atom.max() - atom.min())
        img[m : m + p, n : n + p] = atom

    return img

## DCT denoising

Generate the DCT basis


In [ ]:
D = np.zeros((M, M))
cnt = 0
for i in range(p):
    for j in range(p):
        basis = np.zeros((p, p))
        basis[i, j] = 1
        D[:, cnt] = idct2(basis).flatten()
        cnt = cnt + 1

In [ ]:
D_img = get_dictionary_img(D)
plt.figure(figsize=(8, 8))
plt.imshow(D_img, cmap="gray")

Denoising: set parameters and initialize the variables


Step = 8


In [ ]:
# initialize the estimated image
img_hat = np.zeros_like(img)

# initialize the weight matrix
weights = np.zeros_like(img)

# set the threshold for the Hard Thresholding
tau = 3 * sigma_noise  # Donoho says: sigma * sqrt(2*log(p^2))

# define the step
STEP = p // 2

Step 1 with uniform weights


Perform the denoising patchwise


In [ ]:
img_hat_step8 = np.zeros_like(img)
weights_step8 = np.zeros_like(img)

for i in range(0, imsz[0] - p + 1, STEP):
    for j in range(0, imsz[1] - p + 1, STEP):
        s = noisy_img[i : i + p, j : j + p]
        x = dct2(s)
        x_HT = np.where(np.abs(x) < tau, 0, x)
        x_HT[0, 0] = x[0, 0]
        s_hat = idct2(x_HT)
        w = 1.0
        img_hat_step8[i : i + p, j : j + p] += w * s_hat
        weights_step8[i : i + p, j : j + p] += w

# Normalize the estimated image with the computed weights, i.e. compute averages
img_hat_step8 = img_hat_step8 / (weights_step8 + 1e-8)
# Compute PSNR
psnr_step8 = 10 * np.log10(1 / np.mean((img_hat_step8 - img) ** 2))

In [ ]:
STEP = 1
img_hat_step1_uniform = np.zeros_like(img)
weights_step1_uniform = np.zeros_like(img)

for i in range(0, imsz[0] - p + 1, STEP):
    for j in range(0, imsz[1] - p + 1, STEP):
        s = noisy_img[i : i + p, j : j + p]
        x = dct2(s)
        x_HT = np.where(np.abs(x) < tau, 0, x)
        x_HT[0, 0] = x[0, 0]
        s_hat = idct2(x_HT)
        w = 1.0
        img_hat_step1_uniform[i : i + p, j : j + p] += w * s_hat
        weights_step1_uniform[i : i + p, j : j + p] += w

img_hat_step1_uniform = img_hat_step1_uniform / (weights_step1_uniform + 1e-8)
psnr_step1_uniform = 10 * np.log10(1 / np.mean((img_hat_step1_uniform - img) ** 2))

Step size 1 with sparsity-aware weights


In [ ]:
img_hat_step1_sparse = np.zeros_like(img)
weights_step1_sparse = np.zeros_like(img)

for i in range(0, imsz[0] - p + 1, STEP):
    for j in range(0, imsz[1] - p + 1, STEP):
        s = noisy_img[i : i + p, j : j + p]
        x = dct2(s)
        x_HT = np.where(np.abs(x) < tau, 0, x)
        x_HT[0, 0] = x[0, 0]
        s_hat = idct2(x_HT)
        # Sparsity-aware weight: number of non-zero coefficients after thresholding
        w = np.sum(x_HT != 0) / M
        img_hat_step1_sparse[i : i + p, j : j + p] += w * s_hat
        weights_step1_sparse[i : i + p, j : j + p] += w

img_hat_step1_sparse = img_hat_step1_sparse / (weights_step1_sparse + 1e-8)
psnr_step1_sparse = 10 * np.log10(1 / np.mean((img_hat_step1_sparse - img) ** 2))


In [ ]:
# Plot the results
fig, ax = plt.subplots(1, 3, figsize=(20, 6))

ax[0].imshow(img_hat_step8, cmap="gray")
ax[0].set_title(f"Estimated Image (step: 8), PSNR = {psnr_step8:.2f}")

ax[1].imshow(img_hat_step1_uniform, cmap="gray")
ax[1].set_title(
    f"Estimated Image (step: 1) with Uniform weights, PSNR = {psnr_step1_uniform:.2f}"
)

ax[2].imshow(img_hat_step1_sparse, cmap="gray")
ax[2].set_title(
    f"Estimated Image (step: 1) with Sparsity-aware weights, PSNR = {psnr_step1_sparse:.2f}"
)

plt.tight_layout()
plt.show()

## Wiener Filtering

Initialize the estimated image via Wiener Filtering


In [ ]:
img_hat_wiener = np.zeros_like(img)
weights = np.zeros_like(img)

Perform the denoising patch wise by wiener filtering


In [ ]:
for i in range(0, imsz[0] - p + 1, STEP):
    for j in range(0, imsz[1] - p + 1, STEP):
        # extract the patch from the noisy image with the top left corner at pixel (ii, jj)
        s = noisy_img[i : i + p, j : j + p]

        # compute the representation w.r.t. the 2D DCT dictionary
        x = dct2(s)

        # extract the patch from the image estimated by HT with the top left corner at pixel (ii, jj)
        s_hat_HT = img_hat_step1_sparse[i : i + p, j : j + p]

        # perform the Wiener filtering (do not filter the DC!)
        x_hat_HT = dct2(s_hat_HT)
        x_wie = x.copy()
        x_wie[1:, :] = (
            x_hat_HT[1:, :] ** 2 / (x_hat_HT[1:, :] ** 2 + sigma_noise**2)
        ) * x[1:, :]
        x_wie[:, 1:] = (
            x_hat_HT[:, 1:] ** 2 / (x_hat_HT[:, 1:] ** 2 + sigma_noise**2)
        ) * x[:, 1:]

        # perform the reconstruction
        s_hat_wie = idct2(x_wie)

        # use uniform weights to aggregate the multiple estimates
        w = 1

        # put the denoised patch into the denoised image using the computed weight
        img_hat_wiener[i : i + p, j : j + p] += w * s_hat_wie

        # store the weight of the current patch in the weight matrix
        weights[i : i + p, j : j + p] += w

# Normalize the estimated image with the computed weights
img_hat_wiener = img_hat_wiener / (weights + 1e-8)

Compute the PSNR of the two estimates


In [ ]:
psnr_wiener = 10 * np.log10(1 / np.mean((img_hat_wiener - img) ** 2))
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
ax[0].imshow(img_hat_step1_uniform, cmap="gray")
ax[0].set_title(f"HT Estimate, PSNR = {psnr_step1_uniform:.2f}")

ax[1].imshow(img_hat_wiener, cmap="gray")
ax[1].set_title(f"Wiener Estimate, PSNR = {psnr_wiener:.2f}")
